In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from io import BytesIO

from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket('capstone_p')
blob = bucket.blob('yelp_business_merged_preprocessed.csv')
path = "gs://capstone_p/yelp_business_merged_preprocessed.csv"

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix as cm,classification_report
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
np.random.seed(500)
data_com_buisness = pd.read_csv(path,encoding='utf-8')


In [4]:
import nltk

wpt = nltk.WordPunctTokenizer()

def preprocess(sent):
  text = wpt.tokenize(sent)
  text_tagged = nltk.pos_tag(text)
  new_text = []
  for word in text_tagged:
      new_text.append(word[0] + "/" + word[1])
    
  doc = ' '.join(new_text)
  return doc

In [6]:
data_com_buisness = data_com_buisness.head(200000)

In [7]:
sent = data_com_buisness['text_1'].apply(preprocess)

In [8]:
sent.iloc[20:25]

20    go/VB year/NN still/RB favorit/JJ place/NN ./....
21    came/VBD brother/RB atmospher/JJ nice/NN ./. g...
22    sit/NN want/VBP wait/NN leav/NN draft/NN revie...
23    eat/NN couch/JJ loung/NN comfi/NN recliners/NN...
24    came/VBD friday/JJ night/NN watch/NN indian/JJ...
Name: text_1, dtype: object

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vec_transformed = CountVectorizer(analyzer='word',token_pattern='(?u)')
data = vec_transformed.fit_transform(sent)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(data)

df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vec_transformed.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
,1.0


In [14]:
# count matrix
count_vector=vec_transformed.transform(data_com_buisness['text_1'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

tf_idf_vector.shape
feature_names = vec_transformed.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
brick,0.366105
alway,0.331495
polit,0.311215
beautiful,0.304133
summer,0.291106
...,...
fall,0.000000
fame,0.000000
famili,0.000000
familiar,0.000000


In [15]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(sent,data_com_buisness['stars_y'],test_size=0.3)

In [16]:
# count matrix
count_vector=vec_transformed.transform(Train_X)
 
# tf-idf scores
Train_X_Tfidf=tfidf_transformer.transform(count_vector)

# count matrix
count_vector=vec_transformed.transform(Test_X)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

In [18]:
from sklearn.metrics import confusion_matrix as cm,classification_report
import sklearn.metrics as metrics
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
#print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print(cm(Test_Y, predictions_NB))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y, predictions_NB))
print("Naive Bayes Accuracy Score using count vect ->",metrics.accuracy_score(Test_Y, predictions_NB))

[[ 4293   347   281   646   974]
 [ 1555   281   804  1732  1326]
 [  764    73  1052  3961  2494]
 [  310     8   344  5501  9735]
 [  296     0    42  1909 21272]]


Classification Report 
              precision    recall  f1-score   support

           1       0.59      0.66      0.62      6541
           2       0.40      0.05      0.09      5698
           3       0.42      0.13      0.19      8344
           4       0.40      0.35      0.37     15898
           5       0.59      0.90      0.72     23519

    accuracy                           0.54     60000
   macro avg       0.48      0.42      0.40     60000
weighted avg       0.50      0.54      0.48     60000

Naive Bayes Accuracy Score using count vect -> 0.5399833333333334


In [20]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf, Train_Y)
y_pred = logreg.predict(Test_X_Tfidf)
print(cm(Test_Y, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y, y_pred))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y, y_pred))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[ 4712   900   411   218   300]
 [ 1608  1566  1416   721   387]
 [  538   946  2925  3006   929]
 [  183   210  1290  7754  6461]
 [  192    70   271  3611 19375]]


Classification Report 
              precision    recall  f1-score   support

           1       0.65      0.72      0.68      6541
           2       0.42      0.27      0.33      5698
           3       0.46      0.35      0.40      8344
           4       0.51      0.49      0.50     15898
           5       0.71      0.82      0.76     23519

    accuracy                           0.61     60000
   macro avg       0.55      0.53      0.53     60000
weighted avg       0.59      0.61      0.59     60000

log reg  Accuracy Score -> 0.6055333333333334


In [ ]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm(Test_Y, y_pred), classes=['1', '2','3','4','5'], title='Confusion matrix')

In [29]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(Train_X_Tfidf, Train_Y)
y_pred_svc = model.predict(Test_X_Tfidf)
print(cm(Test_Y, y_pred_svc))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y, y_pred_svc))
print("linear SVC Accuracy Score  Count vect->",metrics.accuracy_score(Test_Y, y_pred_svc))

[[ 4970   694   353   223   301]
 [ 1778  1423  1312   760   425]
 [  635   824  2714  3153  1018]
 [  220   176  1057  7639  6806]
 [  190    58   222  3263 19786]]


Classification Report 
              precision    recall  f1-score   support

           1       0.64      0.76      0.69      6541
           2       0.45      0.25      0.32      5698
           3       0.48      0.33      0.39      8344
           4       0.51      0.48      0.49     15898
           5       0.70      0.84      0.76     23519

    accuracy                           0.61     60000
   macro avg       0.55      0.53      0.53     60000
weighted avg       0.59      0.61      0.59     60000

linear SVC Accuracy Score  Count vect-> 0.6088666666666667


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
vec_transformed = CountVectorizer(min_df=200, max_df=150.0, lowercase=True,analyzer='word',ngram_range=(1, 2))
data = vec_transformed.fit_transform(sent)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(data)

df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vec_transformed.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
nn,1.000105
jj,1.005701
rb,1.201473
vb,1.211473
nns,1.321996
...,...
cupcakes,7.902773
cleaning nn,7.902773
mouss nn,7.902773
girl nns,7.902773


In [31]:
# count matrix
count_vector=vec_transformed.transform(data_com_buisness['text_1'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

tf_idf_vector.shape
feature_names = vec_transformed.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
brick,0.366105
alway,0.331495
polit,0.311215
beautiful,0.304133
summer,0.291106
...,...
jj friends,0.000000
jj fries,0.000000
jj front,0.000000
jj frozen,0.000000


In [32]:
# count matrix
count_vector=vec_transformed.transform(Train_X)
 
# tf-idf scores
Train_X_Tfidf=tfidf_transformer.transform(count_vector)

# count matrix
count_vector=vec_transformed.transform(Test_X)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

In [33]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(Train_X_Tfidf, Train_Y)
y_pred_svc = model.predict(Test_X_Tfidf)
print(cm(Test_Y, y_pred_svc))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y, y_pred_svc))
print("linear SVC Accuracy Score  Count vect->",metrics.accuracy_score(Test_Y, y_pred_svc))

[[ 4737   911   376   219   298]
 [ 1678  1551  1386   704   379]
 [  627  1005  2896  2822   994]
 [  212   268  1363  7423  6632]
 [  177   109   284  3656 19293]]


Classification Report 
              precision    recall  f1-score   support

           1       0.64      0.72      0.68      6541
           2       0.40      0.27      0.33      5698
           3       0.46      0.35      0.40      8344
           4       0.50      0.47      0.48     15898
           5       0.70      0.82      0.75     23519

    accuracy                           0.60     60000
   macro avg       0.54      0.53      0.53     60000
weighted avg       0.58      0.60      0.58     60000

linear SVC Accuracy Score  Count vect-> 0.5983333333333334


In [34]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf, Train_Y)
y_pred = logreg.predict(Test_X_Tfidf)
print(cm(Test_Y, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y, y_pred))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y, y_pred))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[[ 4030  1245   581   326   359]
 [ 1513  1676  1414   723   372]
 [  748  1245  2772  2596   983]
 [  420   555  1807  7082  6034]
 [  493   273   549  4220 17984]]


Classification Report 
              precision    recall  f1-score   support

           1       0.56      0.62      0.59      6541
           2       0.34      0.29      0.31      5698
           3       0.39      0.33      0.36      8344
           4       0.47      0.45      0.46     15898
           5       0.70      0.76      0.73     23519

    accuracy                           0.56     60000
   macro avg       0.49      0.49      0.49     60000
weighted avg       0.55      0.56      0.55     60000

log reg  Accuracy Score -> 0.5590666666666667
